# 2MRS Group Catalogs

In this notebook we will explore the various 2MRS group catalogs and look at the most massive halos. We'll then compile the group catalogs to be used in the data analysis.

In [2]:
import os, sys
from itertools import compress

import numpy as np
import pandas as pd
import healpy as hp
import matplotlib.pyplot as plt
from astropy.cosmology import Planck13, Planck15, z_at_value
import scipy.interpolate as ip
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.interpolate import InterpolatedUnivariateSpline
from astropy.io import fits
from tqdm import *
from astropy import units as u
from astropy.coordinates import SkyCoord, Distance

from units import *
# import tools
import halo as hl
import hmf

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
work_dir = '/group/hepheno/smsharma/Fermi-LSS/'

Load the fermi plugin. This is just to get the PS mask to see which clusters are masked, but we don't need this anymore.

In [4]:
# # Load the fermi plugin
# # This is just to get the PS mask

# import fermi.fermi_plugin as fp
# from global_variables import *
# work_dir = '/group/hepheno/smsharma/Fermi-LSS/'
# reload(fp)

# nside=512
# eventclass=5 # 2 (Source) or 5 (UltracleanVeto)
# eventtype=0 # 0 (all), 3 (bestpsf) or 5 (top3 quartiles)
# mask_type='top300'
# force_mask_at_bin_number=10
# emin_bin=0
# emax_bin=40
# band_mask_val = 30

# f_global = fp.fermi_plugin(maps_dir,fermi_data_dir=fermi_data_dir,work_dir=work_dir,CTB_en_min=emin_bin,CTB_en_max=emax_bin,nside=nside,eventclass=eventclass,eventtype=eventtype,newstyle=1,data_July16=True)
# f_global.make_ps_mask(mask_type = mask_type, energy_bin = force_mask_at_bin_number,force_energy=False)

## Huchra et al catalog (original 2MRS)

Huchra (original 2MASS) catalog. We don't actually do anything with it, just look at the 2MASS IDs etc to confirm the format.

In [5]:
# fits.open("../../../AdditionalData/catalog/2mrs_1175_done.fits")[1].data['TMID']

## Cosmicflow

Cosmicflow catalog. Has a bunch of info that we use like common cluster names.

In [6]:
from astropy.io import ascii
cf_df = ascii.read("../DataFiles/Misc//ajaa259at2_mrt.txt").to_pandas()
cf_df.columns.values

array(['Nest', 'o_<DM>-gp', '<DM>-gp', 'e_<DM>-gp', '<Dist>-gp', 'Abell',
       'GName', 'Npv', 'PGC1', 'GLON-gp', 'GLAT-gp', 'SGLON-gp',
       'SGLAT-gp', 'logLKs', 'cf', 'Vsigma-proj', 'R2t-proj',
       '<Vhelio>-gp', '<Vgsr>-gp', '<Vls>-gp', '<Vcmb>-g', '<Vcmba>-gp',
       'Vrms-gp', 'Mvir', 'Mlum', 'LDC', 'HDC', '2M++', 'MK2011', 'Icnt'], dtype=object)

## Tully catalog

Do some data munging to match Tully 2MRS and Tully clusters.

In [7]:
# Tully version of 2MRS table
twomrs_df =  pd.read_csv("../DataFiles/Misc//2MRS1175.csv",delimiter="|")
twomrs_df.head()

,pgc,ID_2MASXJ,RAJ,DEJ,Glon,Glat,SGL,SGB,K_c,H_c,...,r_ext,b/a,Flgs,Type,So,Vhel,e_V,C,Bib_z,Name
0,1,23595879+0042066,359.99500,0.70179,96.87975,-59.54168,293.35040,13.42997,11.503,11.882,...,1.473,0.72,0000,98,NN,24478,32,F,20112MRS.FLWO.0000H,23595879+0042066
1,2,00000168+4716282,0.00695,47.27456,113.95532,-14.69903,341.64417,20.73886,9.683,9.969,...,2.001,0.60,0333,3B_T,ZC,5017,9,N,1991RC3.9.C...0000d,UGC_12889
2,5,00000914+3244182,0.03814,32.73844,110.62066,-28.90421,326.17715,19.78053,10.608,10.912,...,1.461,0.80,0000,-4,ZC,10372,19,N,1999PASP..111..438F,IC_5370
3,12,00000865-0622263,0.03601,-6.37399,90.19206,-65.93002,286.42487,11.35101,11.224,11.497,...,1.609,0.26,0000,4,ZC,6531,45,6,20096dF...C...0000J,g0000086-062226
4,14,00000701+0816448,0.02926,8.27913,101.78542,-52.47268,300.91431,15.36726,10.758,11.066,...,1.602,0.60,0666,-5,ZC,11602,0,N,2000ApJS..129..547B,UGC_12890


In [8]:
# North clusters
twomrs_ng_clusters_df = pd.read_csv("../DataFiles/Misc/2MRS1175NG.csv",delimiter="|")

In [9]:
# South clusters
twomrs_sg_clusters_df = pd.read_csv("../DataFiles/Misc/2MRS1175SG.csv",delimiter="|")

In [10]:
# Combine north and south cluster catalogs
twomrs_g_clusters_df = pd.concat([twomrs_ng_clusters_df, twomrs_sg_clusters_df])

In [11]:
# Get unique galaxy groups
twomrs_g_clusters_Nests = pd.Series.unique(twomrs_g_clusters_df['Nest'])
twomrs_g_clusters_PGC1 = []

# Loop over unique galaxy groups and get the brightest central
for Nest in tqdm(twomrs_g_clusters_Nests):
    twomrs_g_clusters_PGC1.append(twomrs_g_clusters_df[twomrs_g_clusters_df['Nest'] == Nest]['PGC1'].values[0])

100%|██████████| 25486/25486 [00:16<00:00, 1541.18it/s]


In [12]:
# Get relevant cluster property arrays and also cross-match with Tully 2MRS to get names of brightets clusters (this is what is most slow)

twomrs_g_clusters_masses = []
twomrs_g_clusters_redshifts = []
twomrs_g_clusters_Glon = []
twomrs_g_clusters_Glat = []
twomrs_g_clusters_name = []
twomrs_g_clusters_rvir = []
twomrs_g_clusters_nmem = []
twomrs_g_clusters_twomasxj = []
twomrs_g_clusters_bibz = []
cf_GName = []

for PGC1 in tqdm(twomrs_g_clusters_PGC1):
    PGC1_loc = twomrs_g_clusters_df['pgc'] == PGC1
    PGC1_loc_cf = cf_df['PGC1'] == PGC1
    if np.sum(PGC1_loc) == 0: continue
    twomrs_g_clusters_nmem.append(np.sum(twomrs_g_clusters_df['PGC1'] == PGC1))

    twomrs_g_clusters_masses.append(twomrs_g_clusters_df[PGC1_loc]['L_Mass12'].values[0]*10**12)
    
    twomrs_g_clusters_redshifts.append(twomrs_g_clusters_df[PGC1_loc]['Vgp'].values[0]/300000.)
    twomrs_g_clusters_Glon.append(twomrs_g_clusters_df[PGC1_loc]['GLong'].values[0])
    twomrs_g_clusters_Glat.append(twomrs_g_clusters_df[PGC1_loc]['GLat'].values[0])
    twomrs_g_clusters_rvir.append(twomrs_g_clusters_df[PGC1_loc]['R2t'].values[0])
    twomrs_g_clusters_name.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Name'].values[0])
    twomrs_g_clusters_twomasxj.append(twomrs_df[twomrs_df['pgc'] == PGC1]['ID_2MASXJ'].values[0])
    twomrs_g_clusters_bibz.append(twomrs_df[twomrs_df['pgc'] == PGC1]['Bib_z'].values[0])
    GName = cf_df[PGC1_loc_cf]['GName'].values
    if len(GName) != 0:
        cf_GName.append(cf_df[PGC1_loc_cf]['GName'].values[0])
    else:
        cf_GName.append(np.nan)

100%|██████████| 25486/25486 [02:54<00:00, 146.15it/s]


Now loop over halos to get J-factors, cvirs etc from `halo.py`.

In [13]:
# hm_noboost = hl.HaloModel(boost_model='None', concentration_model='correa')
hm_bartels = hl.HaloModel(boost_model='bartels', concentration_model='duffy')

Loading bartels


In [13]:
NSIDE = 512
NPIX = hp.nside2npix(NSIDE)

Jfactor_ary = []
Jfactorp1_ary = []
Jfactorm1_ary = []

twomrs_g_clusters_rvir_der = []
twomrs_g_clusters_cvir_der = []

# maskatloc=[]

# Interpolated cvir errors
cvir_err_arys = np.load("/group/hepheno/smsharma/Fermi-LSS/AdditionalData/cvir_err.npy")
cvir_err_inter = InterpolatedUnivariateSpline(cvir_err_arys[0],cvir_err_arys[1])

for i in tqdm(range(len(twomrs_g_clusters_masses))):
    
    l, b = twomrs_g_clusters_Glon[i], twomrs_g_clusters_Glat[i]
    
    mvir = twomrs_g_clusters_masses[i]*M_s
    z = twomrs_g_clusters_redshifts[i]

    Jfactor_true = hm_bartels.Jfactor(mvir, z)
    
    cvir_err = cvir_err_inter(np.log10(mvir/M_s))
    cvir = hm_bartels.cvir
    cvir_up = 10**(np.log10(cvir) + cvir_err*np.log10(cvir))
    cvir_down = 10**(np.log10(cvir) - cvir_err*np.log10(cvir))
    
    Jfactorp1_true = hm_bartels.Jfactor(mvir + .2*mvir, z,cvir_up)
    Jfactorm1_true = hm_bartels.Jfactor(mvir - .2*mvir, z,cvir_down)

    twomrs_g_clusters_rvir_der.append(hm_bartels.rvir)
    twomrs_g_clusters_cvir_der.append(cvir)

    Jfactor_ary.append(Jfactor_true)
    Jfactorp1_ary.append(Jfactorp1_true)
    Jfactorm1_ary.append(Jfactorm1_true)          

#     maskatloc.append((f_global.ps_mask_array[14] > 0)[loc])

    hm_bartels.cvir = -999
    hm_bartels.rvir = -999

100%|██████████| 25485/25485 [00:07<00:00, 3538.09it/s]


Make dataframe and add in information we just inferred

In [14]:
nhalos = 100

sales = zip(twomrs_g_clusters_name,\
            np.log10(np.array(Jfactor_ary)/(GeV**2*Centimeter**-5)),\
            np.log10(np.array(Jfactorp1_ary)/(GeV**2*Centimeter**-5)),\
            np.log10(np.array(Jfactorm1_ary)/(GeV**2*Centimeter**-5)),\
            np.log10(twomrs_g_clusters_masses),twomrs_g_clusters_redshifts,\
            twomrs_g_clusters_Nests,twomrs_g_clusters_nmem, \
            twomrs_g_clusters_Glon, twomrs_g_clusters_Glat,\
            twomrs_g_clusters_cvir_der, np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc),\
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc)/np.array(twomrs_g_clusters_cvir_der),\
            twomrs_g_clusters_twomasxj,
            twomrs_g_clusters_bibz,
            cf_GName)
labels = ['Name',"logJ_inf","logJ+_inf","logJ-_inf","logMvir_inf", 'z','Nest',"Ng","l","b","cvir_inf","rvir_inf","rs", "ID_2MASXJ", "Bib_z","GName"]
df = pd.DataFrame.from_records(sales, columns=labels)
df = df[df['z'] < 0.03] # Cut to avoid the issue with crazy massive halos from overincluding satellite contribution
dftop = df.sort(['logJ_inf'],ascending=False)[:nhalos]

/group/hepheno/heptools/anaconda3/envs/venv_py27/lib/python2.7/site-packages/ipykernel/__main__.py:9: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


Add 3FGL-related information

In [15]:
# Load in the 3FGL catalog 
source_3fg_df = pd.read_csv('/group/hepheno/smsharma/Fermi-High-Latitudes/Fermi-HighLat/sim_new/data/3fgl.dat', sep='|', comment='#')

In [16]:
# Remove whitespace from column names
source_3fg_df.rename(columns=lambda x: x.strip(), inplace=True)
for col in source_3fg_df.columns.values:
    try:
        source_3fg_df[col] = source_3fg_df[
            col].map(str.strip)
    except TypeError:
        continue

# Convert to numeric data
source_3fg_df = source_3fg_df.convert_objects(
    convert_numeric=True)

/group/hepheno/heptools/anaconda3/envs/venv_py27/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [17]:
# Astropy-formatted coordinates of cluster and 3FGL
c2 = SkyCoord("galactic", l=dftop['l']*u.deg, b=dftop['b']*u.deg)
c3 = SkyCoord("galactic", l=source_3fg_df['_Lii']*u.deg, b=source_3fg_df['_Bii']*u.deg)

In [18]:
# Select 3FGL sources within some radius

threefgl0p5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, .5*u.deg)
for i in range(len(dftop)):
    threefgl0p5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl1deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 1*u.deg)
for i in range(len(dftop)):
    threefgl1deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl2deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 2*u.deg)
for i in range(len(dftop)):
    threefgl2deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)
    
threefgl5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 5*u.deg)
for i in range(len(dftop)):
    threefgl5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

In [19]:
# 3FGL sources within x degrees
dftop['3FGL 0.5'] = pd.Series(threefgl0p5deg, index=dftop.index)
dftop['3FGL 1'] = pd.Series(threefgl1deg, index=dftop.index)
dftop['3FGL 2'] = pd.Series(threefgl2deg, index=dftop.index)
dftop['3FGL 5'] = pd.Series(threefgl5deg, index=dftop.index)

In [20]:
# Number of 3FGL sources within 5 degrees
dftop['N3FGL 5'] = pd.Series([len(x) for x in threefgl5deg], index=dftop.index)

In [21]:
# Angular extension

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=hm_bartels.h*100, Om0=hm_bartels.omega_m)
ang_ext = 2*dftop['rs'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)
rvir_ang = 2*dftop['rvir_inf'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)


dftop['ang_ext'] = pd.Series(ang_ext, index=dftop.index)
# dftop['rvir_ang'] = pd.Series(rvir_ang, index=dftop.index)

In [22]:
# M200
dftop['logM200_inf'] = pd.Series(np.log10(np.array([hm_bartels.M200_cvir(m, z, c) for m,z,c in zip(10**dftop['logMvir_inf'].values*M_s, dftop['z'], dftop['cvir_inf'])])/M_s), index=dftop.index)

In [23]:
# Boost factor (1+bsh)
dftop['bsh'] = pd.Series([1+hm_bartels.bsh(m, z).flatten()[0] for m,z in zip(10**dftop['logMvir_inf']*M_s,dftop['z'])], index=dftop.index)

In [24]:
dftop.columns.values

array(['Name', 'logJ_inf', 'logJ+_inf', 'logJ-_inf', 'logMvir_inf', 'z',
       'Nest', 'Ng', 'l', 'b', 'cvir_inf', 'rvir_inf', 'rs', 'ID_2MASXJ',
       'Bib_z', 'GName', '3FGL 0.5', '3FGL 1', '3FGL 2', '3FGL 5',
       'N3FGL 5', 'ang_ext', 'logM200_inf', 'bsh'], dtype=object)

In [25]:
# dftop.to_csv("2MRSTully_top100_DATAPAPER_everything_custom_v2.csv")

In [26]:
dftop.head()

,Name,logJ_inf,logJ+_inf,logJ-_inf,logMvir_inf,z,Nest,Ng,l,b,...,Bib_z,GName,3FGL 0.5,3FGL 1,3FGL 2,3FGL 5,N3FGL 5,ang_ext,logM200_inf,bsh
4387,MESSIER_049,18.772818,19.073414,18.474610,14.723456,0.004970,100002,192,286.9222,70.1960,...,2000MNRAS.313..469S,Virgo,[],[],"[3FGL J1226.8+0638, 3FGL J1223.3+0818]","[3FGL J1222.4+0414, 3FGL J1239.5+0443, 3FGL J1...",8,2.041844,14.622280,5.616094
5221,NGC_5128,18.314975,18.652306,17.982843,13.658011,0.002617,100350,6,309.5177,19.4181,...,1978PASP...90..237G,NaN,[3FGL J1325.4-4301],"[3FGL J1324.0-4330e, 3FGL J1325.4-4301]","[3FGL J1333.8-4417, 3FGL J1324.0-4330e, 3FGL J...","[3FGL J1328.5-4728, 3FGL J1326.7-4727, 3FGL J1...",14,1.397985,13.566698,4.585872
15807,NGC_1316,18.205750,18.523302,17.880881,14.025306,0.004540,200015,49,240.1629,-56.6900,...,1998A&AS..130..267L,Fornax,[3FGL J0322.5-3721],[3FGL J0322.5-3721],[3FGL J0322.5-3721],"[3FGL J0334.3-4008, 3FGL J0334.3-3726, 3FGL J0...",4,1.147493,13.930842,5.143022
17658,ESO_137-_G_006,18.086284,18.365223,17.808708,15.103804,0.017060,200002,167,325.2889,-7.2138,...,2008MNRAS.383..445W,Norma,[],[],"[3FGL J1630.3-6126, 3FGL J1603.7-6011]","[3FGL J1558.9-6432, 3FGL J1630.3-6126, 3FGL J1...",8,0.872107,14.999911,6.170136
2103,MESSIER_081,18.068941,18.488976,17.659301,11.260071,0.000263,101368,3,142.0917,40.9000,...,1991RC3.9.C...0000d,NaN,[],[3FGL J0955.4+6940],[3FGL J0955.4+6940],"[3FGL J0958.6+6534, 3FGL J0919.4+6604, 3FGL J1...",6,1.404483,11.186874,2.979207


# Lu catalog

Lu catalog -- this is older code so need to revisit to make sure it's fine. At a first glance `h` isn't correctly defined.

In [14]:
# 2MRS Lu group catalog
twomrs_group_lu =  pd.read_csv("../DataFiles/Misc//2MRS_GROUP",sep=r"\s*", header=None, names=['IDG','Mh','z','idm','ID'])
twomrs_group_lu.head()

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,IDG,Mh,z,idm,ID
0,1,14.374027,0.004352,72,72
1,2,14.594560,0.019161,530,530
2,3,14.597363,0.011353,386,386
3,4,14.762754,0.023568,672,672
4,5,14.844658,0.031335,1681,1681


In [15]:
# 2MRS Lu members
twomrs_member_lu =  pd.read_csv("../DataFiles/Misc//2MRS_MEMBER",sep=r"\s*", header=None, names=['IDG','idm','ID'])
twomrs_member_lu.head()

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,IDG,idm,ID
0,1,15,15
1,1,21,21
2,1,25,25
3,1,38,38
4,1,40,40


In [16]:
 # 2MRS Lu
twomrs_lu =  pd.read_csv("../DataFiles/Misc//2MRS",sep=r"\s*", header=None, names=['ID' , 'NAME' ,'RA' ,'DEC' ,'l' ,'b' , 'Ks' , 'H' , 'J' ,'v', 'dit', 'vlg'])
twomrs_lu.head()

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,ID,NAME,RA,DEC,l,b,Ks,H,J,v,dit,vlg
0,1,00424433+4116074,10.68471,41.26875,121.17430,-21.57319,0.797,0.929,1.552,-300.0,0.8,-33.5
1,2,00473313-2517196,11.88806,-25.28880,97.36301,-87.96452,3.815,4.132,4.858,243.0,0.0,276.2
2,3,09553318+6903549,148.88831,69.06526,142.09190,40.90022,3.898,4.131,4.784,-34.0,3.6,107.6
3,4,13252775-4301073,201.36560,-43.01871,309.51639,19.41761,3.948,4.244,4.931,547.0,0.0,300.7
4,5,13052727-4928044,196.36369,-49.46790,305.27151,13.34017,4.471,4.790,5.508,563.0,0.0,298.6


In [17]:
# Assign brightest galaxy names to groups
twomrs_group_lu_name = []
twomrs_group_lu_l = []
twomrs_group_lu_b = []

for i in tqdm(range(len(twomrs_group_lu))):
    dfloc = twomrs_lu[twomrs_lu['ID'] == twomrs_group_lu['ID'][i]]
    if np.sum(twomrs_df['ID_2MASXJ'] == dfloc['NAME'].values[0]) != 0:
        twomrs_group_lu_name.append(twomrs_df[twomrs_df['ID_2MASXJ'] == dfloc['NAME'].values[0]]['Name'].values[0])
    else:
        twomrs_group_lu_name.append('UNKNOWN')
    twomrs_group_lu_l.append(dfloc['l'])
    twomrs_group_lu_b.append(dfloc['b'])

100%|██████████| 29904/29904 [05:12<00:00, 95.84it/s]


In [18]:
h = Planck15.h

In [19]:
import Jpdf

hm_bartels = hl.HaloModel(boost_model='bartels', concentration_model='correa_Planck15', omega_m=Planck15.Om0, omega_lambda=Planck15.Ode0, h = Planck15.h)

Loading bartels
Interpolating concentration for DarkSky
Using model correa_Planck15


In [25]:
mulog10Jfnb_arr = []
sigmalog10Jfnb_arr = []
mulog10Jf_arr = []
sigmalog10Jf_arr = []

twomrs_g_clusters_rvir_der = []
twomrs_g_clusters_cvir_der = []

# Interpolated cvir errors
cvir_err_arys = np.load("../DataFiles/Misc/cvir_err.npy")
cvir_err_inter = InterpolatedUnivariateSpline(cvir_err_arys[0],cvir_err_arys[1])

Ji = Jpdf.Jc(hm_bartels)

for i in tqdm(range(len(twomrs_g_clusters_masses))):
    hm_bartels.cvir = -999
    hm_bartels.rvir = -999
        
    mvir = (10**twomrs_group_lu['Mh'].values[i])*M_s/h # This is in keV...

    # Some clusters have a 0 mass, if so set mvir to be tiny
#     if twomrs_g_clusters_masses[i] < 0.1:
#         mvir = 0.01*M_s
#         continue
    
    z = twomrs_group_lu['z'].values[i]
    
    if z < 0.03 and np.log10(mvir/M_s) > 10.1: 
        
        cvir = hm_bartels.c_vir(mvir, z)
        rvir = hm_bartels.r_vir(mvir, z)

        mvir /= M_s # Msun

        mulog10M = np.log10(mvir)
        sigmalog10M = mulog10M*0.01

        mulog10c = np.log10(cvir)
        #sigmalog10c = 0.15 # NB: Might want to update this later!    
        sigmalog10c = cvir_err_inter(np.log10(mvir))

        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J = Ji.log10Jmusig(mulog10M, sigmalog10M, mulog10c, sigmalog10c, z)
    
    else:
        cvir, rvir = -999., -999.
        mulog10Jnb, sigmalog10Jnb, mulog10J, sigmalog10J =  -999., -999., -999., -999.
        
    mulog10Jfnb_arr.append(mulog10Jnb)
    sigmalog10Jfnb_arr.append(sigmalog10Jnb)
    mulog10Jf_arr.append(mulog10J)
    sigmalog10Jf_arr.append(sigmalog10J)
    
    twomrs_g_clusters_rvir_der.append(rvir)
    twomrs_g_clusters_cvir_der.append(cvir)

100%|██████████| 25485/25485 [04:22<00:00, 181.57it/s]


In [26]:
twomrs_group_lu_l = [twomrs_group_lu_l[i].values[0] for i in range(len(twomrs_group_lu_l))]
twomrs_group_lu_b = [twomrs_group_lu_b[i].values[0] for i in range(len(twomrs_group_lu_l))]

In [27]:
twomrs_group_lu['Mh'].values

array([ 14.374027,  14.59456 ,  14.597363, ...,  12.263852,  12.356584,
        12.710698])

In [28]:
np.log10((10**twomrs_group_lu['Mh'].values)*M_s/h/M_s)

array([ 14.54318181,  14.76371481,  14.76651781, ...,  12.43300681,
        12.52573881,  12.87985281])

In [46]:
nhalos = 10000

sales = zip(twomrs_group_lu_name,\
            mulog10Jfnb_arr,\
            sigmalog10Jfnb_arr,\
            mulog10Jf_arr,\
            sigmalog10Jf_arr,\
            np.log10((10**twomrs_group_lu['Mh'].values)*M_s/h/M_s),
            np.array(twomrs_group_lu['z'].values),
            np.array(twomrs_group_lu_l),
            np.array(twomrs_group_lu_b),
            np.array(twomrs_g_clusters_cvir_der),
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc),
            np.array(twomrs_g_clusters_rvir_der)/(.001*Mpc)/np.array(twomrs_g_clusters_cvir_der))

labels = ["Name", "mulog10Jnb_inf", "siglog10Jnb_inf", "mulog10J_inf", "siglog10J_inf", "log10Mvir_inf", "z",  "l", "b", "cvir_inf", "rvir_inf", "rs"]
df = pd.DataFrame.from_records(sales, columns=labels)
df = df[df["z"] < 0.03] # Cut to avoid the issue with crazy massive halos from overincluding satellite contribution
dftop = df.sort(["mulog10J_inf"],ascending=False)[:nhalos]

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [47]:
# Change Virgo and Fornax centers

dftop.loc[dftop['Name'] == 'NGC_4365','l'] = 283.7777978
dftop.loc[dftop['Name'] == 'NGC_4365','b'] = 74.4911308

dftop.loc[dftop['Name'] == 'NGC_1316','l'] = 236.72
dftop.loc[dftop['Name'] == 'NGC_1316','b'] = -53.64

In [48]:
# Load in the 3FGL catalog 
source_3fg_df = pd.read_csv('../DataFiles/Catalogs//3fgl.dat', sep='|', comment='#')

In [49]:
# Remove whitespace from column names
source_3fg_df.rename(columns=lambda x: x.strip(), inplace=True)
for col in source_3fg_df.columns.values:
    try:
        source_3fg_df[col] = source_3fg_df[
            col].map(str.strip)
    except TypeError:
        continue

# Convert to numeric data
source_3fg_df = source_3fg_df.convert_objects(
    convert_numeric=True)

/tigress/smsharma/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:12: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


In [50]:
# Astropy-formatted coordinates of cluster and 3FGL
c2 = SkyCoord("galactic", l=dftop['l']*u.deg, b=dftop['b']*u.deg)
c3 = SkyCoord("galactic", l=source_3fg_df['_Lii']*u.deg, b=source_3fg_df['_Bii']*u.deg)

In [51]:
# Select 3FGL sources within some radius

threefgl0p5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, .5*u.deg)
for i in range(len(dftop)):
    threefgl0p5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl1deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 1*u.deg)
for i in range(len(dftop)):
    threefgl1deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

threefgl2deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 2*u.deg)
for i in range(len(dftop)):
    threefgl2deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)
    
threefgl5deg = []

idx3fgl, idx2mass, d2d, _ = c2.search_around_sky(c3, 5*u.deg)
for i in range(len(dftop)):
    threefgl5deg.append(source_3fg_df['name'][idx3fgl[idx2mass == i]].values)

In [52]:
# 3FGL sources within x degrees
dftop['3FGL 0.5'] = pd.Series(threefgl0p5deg, index=dftop.index)
dftop['3FGL 1'] = pd.Series(threefgl1deg, index=dftop.index)
dftop['3FGL 2'] = pd.Series(threefgl2deg, index=dftop.index)
#dftop['3FGL 5'] = pd.Series(threefgl5deg, index=dftop.index)

In [53]:
# Number of 3FGL sources within 5 degrees
dftop['N3FGL 5'] = pd.Series([len(x) for x in threefgl5deg], index=dftop.index)

In [54]:
# Angular extension

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=hm_bartels.h*100, Om0=hm_bartels.omega_m)
ang_ext = 2*dftop['rs'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)
rvir_ang = 2*dftop['rvir_inf'].values/(cosmo.angular_diameter_distance(dftop['z']).value*1000)*(180/np.pi)


dftop['ang_ext'] = pd.Series(ang_ext, index=dftop.index)
# dftop['rvir_ang'] = pd.Series(rvir_ang, index=dftop.index)

In [55]:
# M200
dftop['log10M200_inf'] = pd.Series(np.log10(np.array([hm_bartels.M200_cvir(m, z, c) for m,z,c in zip(10**dftop['log10Mvir_inf'].values*M_s, dftop['z'], dftop['cvir_inf'])])/M_s), index=dftop.index)

In [56]:
# Boost factor (1+bsh)
dftop['c200_inf'] = pd.Series([hm_bartels.c200_from_cvir(cvir, z) for cvir,z in zip(dftop['cvir_inf'],dftop['z'])], index=dftop.index)

In [57]:
# c200 from cvir
dftop['bsh'] = pd.Series([1+hm_bartels.bsh(m, z).flatten()[0] for m,z in zip(10**dftop['log10Mvir_inf']*M_s,dftop['z'])], index=dftop.index)

In [58]:
dftop

,Name,mulog10Jnb_inf,siglog10Jnb_inf,mulog10J_inf,siglog10J_inf,log10Mvir_inf,z,l,b,cvir_inf,rvir_inf,rs,3FGL 0.5,3FGL 1,3FGL 2,N3FGL 5,ang_ext,log10M200_inf,c200_inf,bsh
0,NGC_4365,18.112697,0.339667,18.846304,0.350691,14.543182,0.004352,283.777798,74.491131,6.556259,1855.085538,282.948779,[3FGL J1230.9+1224],[3FGL J1230.9+1224],"[3FGL J1223.2+1215, 3FGL J1230.9+1224, 3FGL J1...",7,1.692461,14.456597,4.911059,5.415107
108,MESSIER_105,18.028893,0.349440,18.697513,0.365249,13.724996,0.002251,233.490300,57.632760,8.139839,990.587917,121.696261,[],[],[3FGL J1049.8+1425],5,1.403722,13.647160,6.134701,4.662509
18,NGC_1316,17.951113,0.339780,18.683839,0.350478,14.530812,0.005187,236.720000,-53.640000,6.574400,1837.123402,279.435886,[],[3FGL J0336.9-3622],[3FGL J0336.9-3622],6,1.403817,14.444449,4.926646,5.404137
117,MESSIER_081,17.960362,0.355434,18.589563,0.358984,13.115030,0.001383,142.091900,40.900220,9.456400,620.408421,65.607253,[],[3FGL J0955.4+6940],[3FGL J0955.4+6940],6,1.230398,13.042611,7.154879,4.257957
17,MESSIER_106,17.632954,0.352429,18.275978,0.360072,13.425976,0.002693,138.319900,68.842510,8.761471,787.345627,89.864548,[],[],[],7,0.866896,13.350956,6.618316,4.395663
114,MESSIER_066,17.617328,0.354939,18.248151,0.358952,13.164657,0.002151,241.961590,64.418460,9.338953,644.355652,68.996561,[],[],[],4,0.832747,13.091875,7.065588,4.273891
2,g1250039-412255,17.466375,0.336479,18.220552,0.355392,14.766518,0.011353,302.655210,21.488980,6.150185,2197.630162,357.327505,[],[],[],7,0.826375,14.677871,4.608658,5.677762
5286,NGC_6946,17.574847,0.360687,18.135852,0.367179,12.442230,0.001146,95.718730,11.672890,10.994817,370.203189,33.670702,[],[],[],4,0.761828,12.374791,8.350172,3.639189
95,04161030-5546510,17.473918,0.352059,18.119355,0.360444,13.463764,0.003353,265.631500,-43.691180,8.680705,810.364534,93.352387,[],[],[],5,0.723867,13.388470,6.557153,4.420148
2208,MESSIER_094,17.510179,0.360739,18.070675,0.367100,12.433933,0.001225,123.362100,76.007770,11.013431,367.844931,33.399667,[],[],[],6,0.707030,12.366555,8.364885,3.634929


In [59]:
dftop.to_csv("../DataFiles/Catalogs/2MRSLu_ALL_DATAPAPER_Planck15_v5.csv")

In [61]:
pd.read_csv("../DataFiles/Catalogs/2MRSLu_ALL_DATAPAPER_Planck15_v5.csv")

,Unnamed: 0,Name,mulog10Jnb_inf,siglog10Jnb_inf,mulog10J_inf,siglog10J_inf,log10Mvir_inf,z,l,b,...,rvir_inf,rs,3FGL 0.5,3FGL 1,3FGL 2,N3FGL 5,ang_ext,log10M200_inf,c200_inf,bsh
0,0,NGC_4365,18.112697,0.339667,18.846304,0.350691,14.543182,0.004352,283.777798,74.491131,...,1855.085538,282.948779,['3FGL J1230.9+1224'],['3FGL J1230.9+1224'],['3FGL J1223.2+1215' '3FGL J1230.9+1224' '3FGL...,7,1.692461,14.456597,4.911059,5.415107
1,108,MESSIER_105,18.028893,0.349440,18.697513,0.365249,13.724996,0.002251,233.490300,57.632760,...,990.587917,121.696261,[],[],['3FGL J1049.8+1425'],5,1.403722,13.647160,6.134701,4.662509
2,18,NGC_1316,17.951113,0.339780,18.683839,0.350478,14.530812,0.005187,236.720000,-53.640000,...,1837.123402,279.435886,[],['3FGL J0336.9-3622'],['3FGL J0336.9-3622'],6,1.403817,14.444449,4.926646,5.404137
3,117,MESSIER_081,17.960362,0.355434,18.589563,0.358984,13.115030,0.001383,142.091900,40.900220,...,620.408421,65.607253,[],['3FGL J0955.4+6940'],['3FGL J0955.4+6940'],6,1.230398,13.042611,7.154879,4.257957
4,17,MESSIER_106,17.632954,0.352429,18.275978,0.360072,13.425976,0.002693,138.319900,68.842510,...,787.345627,89.864548,[],[],[],7,0.866896,13.350956,6.618316,4.395663
5,114,MESSIER_066,17.617328,0.354939,18.248151,0.358952,13.164657,0.002151,241.961590,64.418460,...,644.355652,68.996561,[],[],[],4,0.832747,13.091875,7.065588,4.273891
6,2,g1250039-412255,17.466375,0.336479,18.220552,0.355392,14.766518,0.011353,302.655210,21.488980,...,2197.630162,357.327505,[],[],[],7,0.826375,14.677871,4.608658,5.677762
7,5286,NGC_6946,17.574847,0.360687,18.135852,0.367179,12.442230,0.001146,95.718730,11.672890,...,370.203189,33.670702,[],[],[],4,0.761828,12.374791,8.350172,3.639189
8,95,04161030-5546510,17.473918,0.352059,18.119355,0.360444,13.463764,0.003353,265.631500,-43.691180,...,810.364534,93.352387,[],[],[],5,0.723867,13.388470,6.557153,4.420148
9,2208,MESSIER_094,17.510179,0.360739,18.070675,0.367100,12.433933,0.001225,123.362100,76.007770,...,367.844931,33.399667,[],[],[],6,0.707030,12.366555,8.364885,3.634929


In [64]:
pd.read_csv("../DataFiles/Catalogs/old_catalogs/2MRSTully_ALL_DATAPAPER_Planck15_v4.csv")

,Unnamed: 0,Name,mulog10Jnb_inf,siglog10Jnb_inf,mulog10J_inf,siglog10J_inf,log10Mvir_inf,z,Nest,Ng,...,rvir_inf,rs,ID_2MASXJ,Bib_z,GName,3FGL 0.5,3FGL 1,3FGL 2,N3FGL 5,ang_ext
0,4387,MESSIER_049,18.164248,0.337346,18.913682,0.354394,14.723456,0.004958,100002,192,...,2.130007e+03,3.410498e+02,12294679+0800014,2000MNRAS.313..469S,Virgo,[],[],['3FGL J1226.8+0638' '3FGL J1223.3+0818'],8,1.791865e+00
1,5221,NGC_5128,17.864995,0.350136,18.526417,0.363590,13.658011,0.002614,100350,6,...,9.408498e+02,1.136660e+02,13252775-4301073,1978PASP...90..237G,NaN,['3FGL J1325.4-4301'],['3FGL J1324.0-4330e' '3FGL J1325.4-4301'],['3FGL J1333.8-4417' '3FGL J1324.0-4330e' '3FG...,14,1.129411e+00
2,15807,NGC_1316,17.682939,0.345964,18.394158,0.349048,14.025306,0.004531,200015,49,...,1.246570e+03,1.657499e+02,03224178-3712295,1998A&AS..130..267L,Fornax,['3FGL J0322.5-3721'],['3FGL J0322.5-3721'],['3FGL J0322.5-3721'],4,9.525756e-01
3,2103,MESSIER_081,17.882948,0.367027,18.357049,0.370216,11.260071,0.000263,101368,3,...,1.494500e+02,1.082288e+01,09553318+6903549,1991RC3.9.C...0000d,NaN,[],['3FGL J0955.4+6940'],['3FGL J0955.4+6940'],6,1.063957e+00
4,4415,MESSIER_094,17.670560,0.357400,18.276359,0.376261,12.882525,0.001590,120491,3,...,5.189799e+02,5.200627e+01,12505314+4107125,1993A&A...272...63M,NaN,[],[],[],6,8.488050e-01
5,3622,MESSIER_109,17.599168,0.349559,18.265725,0.364692,13.706718,0.003717,100008,56,...,9.763834e+02,1.195206e+02,11573598+5322282,2001A&A...370..765V,UrsaMajor,[],[],[],4,8.363012e-01
6,17658,ESO_137-_G_006,17.419112,0.331870,18.209407,0.335224,15.103804,0.016896,200002,167,...,2.842570e+03,5.071065e+02,16150386-6054261,2008MNRAS.383..445W,Norma,[],[],['3FGL J1630.3-6126' '3FGL J1603.7-6011'],8,7.933490e-01
7,4833,MESSIER_104,17.462935,0.346526,18.168173,0.374774,13.971740,0.005562,100128,15,...,1.196011e+03,1.569356e+02,12395949-1137230,2000MNRAS.313..469S,NaN,['3FGL J1239.1-1158'],['3FGL J1239.1-1158'],['3FGL J1239.1-1158'],5,7.356519e-01
8,15605,NGC_1275,17.375641,0.332121,18.165509,0.335306,15.082785,0.017429,200001,180,...,2.796667e+03,4.962753e+02,03194823+4130420,1992ApJS...83...29S,Perseus,['3FGL J0319.8+4130'],['3FGL J0316.6+4119' '3FGL J0319.8+4130'],['3FGL J0316.6+4119' '3FGL J0319.8+4130'],7,7.531715e-01
9,4768,NGC_4696,17.404204,0.336041,18.161692,0.356260,14.794488,0.012818,100003,113,...,2.244399e+03,3.680181e+02,12484927-4118399,1991RC3.9.C...0000d,Centaurus,[],[],[],7,7.551791e-01


In [66]:
pd.read_csv("../DataFiles/Catalogs/2MRSTully_ALL_DATAPAPER_Planck15_v5.csv")

,Unnamed: 0,Name,mulog10J_inf,siglog10J_inf,mulog10Jnb_inf,siglog10Jnb_inf,mulog10JB_inf,siglog10JB_inf,mulog10JBnb_inf,siglog10JBnb_inf,...,3FGL 1,3FGL 2,3FGL 5,3FGL 10,N3FGL 5,ang_ext,theta_vir,log10M200_inf,ra,dec
0,4387,MESSIER_049,18.987653,0.354419,18.238218,0.337371,18.735315,0.383425,17.985881,0.367725,...,['3FGL J1230.9+1224'],['3FGL J1223.2+1215' '3FGL J1230.9+1224' '3FGL...,['3FGL J1223.3+0818' '3FGL J1223.2+1215' '3FGL...,['3FGL J1222.4+0414' '3FGL J1239.5+0443' '3FGL...,7,1.994681,6.230767,14.634695,187.705935,12.391097
1,2103,MESSIER_081,18.770320,0.370221,18.296219,0.367032,18.577863,0.395983,18.103762,0.393003,...,['3FGL J0955.4+6940'],['3FGL J0955.4+6940'],['3FGL J0958.6+6534' '3FGL J0919.4+6604' '3FGL...,['3FGL J0925.6+5959' '3FGL J1031.6+6021' '3FGL...,6,1.750725,12.088588,11.199220,148.888240,69.065403
2,5221,NGC_5128,18.486783,0.363587,17.825361,0.350133,18.256768,0.392182,17.595346,0.379742,...,['3FGL J1324.0-4330e' '3FGL J1325.4-4301'],['3FGL J1333.8-4417' '3FGL J1324.0-4330e' '3FG...,['3FGL J1328.5-4728' '3FGL J1326.7-4727' '3FGL...,['3FGL J1326.6-5256' '3FGL J1244.3-4955' '3FGL...,14,1.103411,4.566442,13.580861,201.367201,-43.018055
3,4833,MESSIER_104,18.319753,0.374821,17.614516,0.346577,18.083316,0.402686,17.378079,0.376538,...,['3FGL J1239.1-1158'],['3FGL J1239.1-1158'],['3FGL J1241.6-1456' '3FGL J1231.2-1411' '3FGL...,['3FGL J1238.2-1958' '3FGL J1215.9-1926' '3FGL...,5,0.895131,3.413221,13.891584,189.997707,-11.623042
4,4768,NGC_4696,18.288559,0.356354,17.531071,0.336141,18.034385,0.385109,17.276896,0.366485,...,[],[],['3FGL J1238.3-4543' '3FGL J1254.9-4423' '3FGL...,['3FGL J1244.3-4955' '3FGL J1251.0-4943' '3FGL...,7,0.892836,2.725833,14.705442,192.205558,-41.310965
5,4350,MESSIER_106,18.217975,0.358942,17.587881,0.355137,17.997652,0.387561,17.367558,0.384040,...,[],[],['3FGL J1224.6+4332' '3FGL J1231.6+4825' '3FGL...,['3FGL J1203.2+3847' '3FGL J1236.6+3901' '3FGL...,7,0.803580,3.771837,13.070419,184.739927,47.303783
6,15807,NGC_1316,18.205228,0.348998,17.494009,0.345914,17.967639,0.378767,17.256420,0.375926,...,['3FGL J0336.9-3622'],['3FGL J0336.9-3622'],['3FGL J0334.3-4008' '3FGL J0334.3-3726' '3FGL...,['3FGL J0335.3-4459' '3FGL J0334.3-4008' '3FGL...,6,0.784066,2.946446,13.944665,54.615768,-35.452772
7,15605,NGC_1275,18.166188,0.335325,17.376321,0.332140,17.905598,0.365287,17.115731,0.362365,...,['3FGL J0316.6+4119' '3FGL J0319.8+4130'],['3FGL J0316.6+4119' '3FGL J0319.8+4130'],['3FGL J0310.8+3814' '3FGL J0342.2+3857' '3FGL...,['3FGL J0336.5+3210' '3FGL J0325.2+3410' '3FGL...,7,0.770546,2.171633,14.990714,49.951166,41.511588
8,4415,MESSIER_094,18.164351,0.376253,17.558553,0.357391,17.948627,0.403426,17.342829,0.385895,...,[],[],['3FGL J1243.1+3627' '3FGL J1256.9+3649' '3FGL...,['3FGL J1310.6+3222' '3FGL J1258.1+3233' '3FGL...,6,0.763045,3.806780,12.811982,192.720770,41.119887
9,3622,MESSIER_109,18.163130,0.364673,17.496574,0.349540,17.932073,0.393205,17.265516,0.379213,...,[],[],['3FGL J1153.4+4932' '3FGL J1215.0+5002' '3FGL...,['3FGL J1145.8+4425' '3FGL J1231.6+4825' '3FGL...,4,0.760059,3.103712,13.629205,179.399725,53.374401
